<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Breast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline

In [0]:
train_data_dir = '/content/drive/My Drive/Augmented '

In [6]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
batch_size = 32
img_width, img_height = 240, 240
# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 7716 images belonging to 2 classes.
Found 1929 images belonging to 2 classes.


In [0]:
epochs = 20
train_samples = 7716
validation_samples = 1929

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam 
#import pickle
import time

model = Sequential()

model.add(Conv2D(32, (2, 2), input_shape=(240,240,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(128, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
'''
model.add(Conv2D(128, (5, 5), input_shape=(736,459,3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
'''
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 239, 239, 32)      416       
_________________________________________________________________
activation_4 (Activation)    (None, 239, 239, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 119, 119, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 118, 118, 64)      8256      
_________________________________________________________________
activation_5 (Activation)    (None, 118, 118, 64)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 59, 59, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 58, 58, 128)      

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Breast.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
32/32 [==============================] - 941s 29s/step - loss: 0.6177 - acc: 0.7070

Epoch 00001: val_acc improved from -inf to 0.70703, saving model to Breast.h5
32/32 [==============================] - 1866s 58s/step - loss: 0.8388 - acc: 0.6455 - val_loss: 0.6177 - val_acc: 0.7070
Epoch 2/20
32/32 [==============================] - 530s 17s/step - loss: 0.5893 - acc: 0.7246

Epoch 00002: val_acc improved from 0.70703 to 0.72461, saving model to Breast.h5
32/32 [==============================] - 1116s 35s/step - loss: 0.6380 - acc: 0.6846 - val_loss: 0.5893 - val_acc: 0.7246
Epoch 3/20
32/32 [==============================] - 526s 16s/step - loss: 0.5816 - acc: 0.7188

Epoch 00003: val_acc did not improve from 0.72461
32/32 [==============================] - 1103s 34s/step - loss: 0.5684 - acc: 0.7412 - val_loss: 0.5816 - val_acc: 0.7188
Epoch 4/20
 8/32 [======>..................

In [0]:
model.save("Breast.model")

In [0]:
import cv2
import tensorflow as tf
import math
#from google.colab.patches import cv2_imshow

CATEGORIES = ["cancer", "normal"]


def prepare(filepath):
    #IMG_SIZE = 220 # 50 in txt-based
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (736,459))

    return new_array.reshape(-1, 736,459, 3)


model = tf.keras.models.load_model("Breast.h5")

prediction = model.predict([prepare('/content/drive/My Drive/dataset/NORMAL/normal_04-A_0475_1.LEFT_MLO.LJPEG.1.jpg')])
pred_percent = math.floor(prediction[0][0]*100)
print(pred_percent,"%")
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])
'''
if(prediction[0][0]>prediction[0][1]):
  result = prediction[0][0]
else:
  result = prediction[0][1]

print(prediction)
print(result*100)
print(CATEGORIES[int(result)])
'''

In [0]:
normal = 2342
cancer = 2367

if(prediction[0][0]>prediction[0][1]):
  result = prediction[0][0]
else:
  result = prediction[0][1]

print(result)

In [0]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen1 = ImageDataGenerator(rescale=1. / 255)
batch_size = 32
train_data_dir1 = '/content/drive/My Drive/dataset1/' 
# Retrieve images and their classes for train and validation sets
train_generator1 = datagen1.flow_from_directory(
        train_data_dir1,
        target_size=(224, 224),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

In [0]:
import os
import glob
from PIL import Image

image_list = []
i = 0
for filename in glob.glob('/content/drive/My Drive/dataset/NORMAL/*.jpg'): #assuming jpg
    im=filename
    image_list.append(im)
    #print(i)
    i +=1
len(image_list)

In [0]:
from PIL import Image
j=0
for i in range(len(image_list)):
  temp = image_list[i]
  im = Image.open(temp)
  temp = temp.replace("jpg", "png")
  temp = temp.replace("/content/drive/My Drive/dataset/NORMAL/", "")
  temp = "/content/drive/My Drive/PNG/normal/"+temp
  im.save(temp)
  print(j)
  j=j+1

In [0]:
image_list[0]

In [0]:
temp = image_list[0]
temp = temp.replace("jpg", "png")
temp = temp.replace("/content/drive/My Drive/dataset/CANCER/", "")
temp = "/content/drive/My Drive/PNG/cancer/"+temp
temp